## 1. Импорт библиотек

In [ ]:
# !pip install glob2
!pip install -U -q PyDrive

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as sst
import time
import re
import os
from os.path import basename
# import glob2
from datetime import date, datetime, timedelta
import plotly.express as px

# PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import warnings
warnings.filterwarnings('ignore')

from statistics import mode
from itertools import combinations
from scipy.stats import ttest_ind, pearsonr, f_oneway

from datetime import datetime, timedelta, time

from sklearn.preprocessing import PowerTransformer, QuantileTransformer, StandardScaler, RobustScaler, MinMaxScaler, Normalizer                              
from sklearn.feature_selection import f_classif, mutual_info_classif

sns.set(style="darkgrid")
pd.set_option('display.max_rows', 1000) # показывать больше строк
pd.set_option('display.max_columns', 50) # показывать больше колонок

In [ ]:
RANDOM_SEED = 42
CURRENT_DATE = datetime.now()

## 2. Определение функций

In [ ]:
# Функции для выравнивания временного ряда

def upsampl_time (d1, delta):
  # upsamling - повышение разрешения каждые 10 секунд, вставляемая сторока заполняется NAN по другим признакам

    d1['Date/Time'] = pd.to_datetime(d1['Date/Time'])
    d1 = d1.set_index('Date/Time')
    d1 = d1.resample(delta).asfreq()
    d1 = d1.reset_index()
    return d1
def downsampl_time (d1, delta):
  # downsamling - понижение разрешения до 10 секунд, при схлопывании по другим признакам берется максимальное значение

    d1['Date/Time'] = pd.to_datetime(d1['Date/Time'])
    d1 = d1.set_index('Date/Time')
    d1 = d1.resample(delta).max()
    d1 = d1.reset_index()
    return d1


def sec_for_min(d1):
  # проставление "обрезанных" секунд при наличии данных для них 

    i=0
    t_old = pd.to_datetime(0)
    delta = timedelta(seconds = 10)
    for t in d1["Date/Time"]:
        if (t_old.time().second >= 50) and (t_old.time().second <= 59):
                delta = timedelta(seconds = 2)
        else: delta = timedelta(seconds = 10)
        #print(t_old, delta)
        if i !=0:
            if t.time().minute == t_old.time().minute:
                t = (t_old + delta)
                d1["Date/Time"][i]=t
        i += 1
        t_old = t
    return d1

In [ ]:
# # Списки номеров скважин STORE в lbf, SPPA и APRS в bar и kPA

# list_well_lbf_STORE = ['204', '210ST','210ST2','216ST','216ST2','217ST1','217ST2','220ST','234ST2','241','243','247','255','256ST2','256ST3','278','279']
# list_well_bar_SPPA = ['204', '210ST','216ST','247']
# list_well_kpa_SPPA = ['241','243','278','279']
# list_well_bar_APRS = ['204', '210ST','216ST','247']
# list_well_kpa_APRS = ['241','243','278','279']

# # Датасет, который будем строить

# COLUMN_NAMES = ['Date/Time','StuckPipe','TVD','DEPT','CDEPTH','HDTH','BPOS','HKLD','STOR','FLWI','RPM','SPPA','ECD','DLS','INCL','AZIM','GR','APRS','BVEL','RIG_STATE','Stick_Slip_Ratio','StickPercentage','ESD']
# data = pd.DataFrame(columns=COLUMN_NAMES) 

# # Переименуем дублирующиеся колонки

# recolumn = {
#     'APRS_RT': 'APRS',
#     'AZIM_CONT_RT': 'AZIM',
#     'ECD_RT': 'ECD',
#     'ECD_ECO_RT': 'ECD', 
#     'DEPTH': 'DEPT',
#     'INCL_CONT_RT': 'INCL', 
#     'DHAP_DH_ECO_RT': 'APRS',
#     'GR_ARC_RT': 'GR', 
#     'GRMA_ECO_RT': 'GR', 
#     'RIG_STATE_EXACT': 'RIG_STATE', 
#     'STICKRATIO': 'Stick_Slip_Ratio'    
#     }

In [ ]:
# Загрузка данных с предобработкой
# n=0
# print(n)
# for dirname, _, filenames in os.walk ("C:/Users/shers/Desktop/SF/Буровая/data"):
#     #print(filenames)
#     for file in filenames:
#         data_f = pd.read_excel("C:/Users/shers/Desktop/SF/Буровая/data/" + file)
#         display(file)
#        # print(data_f.head(2))
#         data_f['file'] = file
#         data_f.rename(columns=recolumn, inplace=True)
#         hole = file.split('-')[0] # добавляем номер буровой
#         #приводим интервал времени к единому
#         max_delta=data_f['Date/Time'].diff().max().seconds 
#         min_delta=data_f['Date/Time'].diff().min().seconds
#         if (max_delta == 60) and (min_delta ==0):
#             #print ('60')
#             data_f = sec_for_min(data_f)
#             data_f = downsampl_time (data_f, '10S')
#         elif (max_delta == 30) and (min_delta ==0):
#             #print =  ( '30')
#             data_f = sec_for_min(data_f)
#             data_f = upsampl_time (data_f, '10S')
#         elif max_delta == 10:
#             #print ( '10')
#             data_f = downsampl_time (data_f, '10S')
#         else: 
#             #print ( '2')        
#             data_f = downsampl_time (data_f, '10S')
       
#         # обрабатываем значения -999.25
#         for col in data_f.columns[data_f.dtypes == 'float64']: 
#             data_f[col]=data_f[col].apply(lambda n: np.nan if n == -999.25 else n).astype('float')
    
#         # приводим к общим единицам измерения
#         data_f['hole']=hole
#         if hole in list_well_lbf_STORE:
#             data_f['STOR'] = data_f['STOR'].astype('float').apply(lambda x: x*0.737 if x != -999.25 else np.nan)
#         if hole in list_well_bar_SPPA:
#             data_f['APRS'] = data_f['APRS'].astype('float').apply(lambda x: x*1.02 if x != -999.25 else np.nan)
#             data_f['SPPA'] = data_f['SPPA'].astype('float').apply(lambda x: x*1.02 if x != -999.25 else np.nan)
#         if hole in list_well_kpa_SPPA:
#             data_f['APRS'] = data_f['APRS'].astype('float').apply(lambda x: x*0.01 if x != -999.25 else np.nan)
#             data_f['SPPA'] = data_f['SPPA'].astype('float').apply(lambda x: x*0.01 if x != -999.25 else np.nan)
        
#         # Замена NaN
#         data_f['StuckPipe'] = data_f['StuckPipe'].fillna(0)

#         data_f[['DEPT', 'HDTH']] = data_f[['DEPT', 'HDTH']].interpolate()
#         data_f['DDEPT'] = data_f['DEPT'].diff() # Новый признак изменения к предыдущему глубины долота
#         if hole == '210ST2':
#             data['RIG_STATE'] = data['RIG_STATE'].interpolate(method='pad')
#             data_f['BPOS'] = data_f['BPOS'].interpolate()
#             data_f.loc[((data_f['BPOS'].isna())&(data_f['RIG_STATE']==2)), 'BPOS'] = 1
#             data_f['HKLD'] = data_f['HKLD'].interpolate()
#             data_f['STOR'] = data_f['STOR'].interpolate() 
        
#         if hole != '210ST2':
#             data_f.loc[((data_f['RIG_STATE'].isna())&(data_f['FLWI']>15)&(data_f['RPM']>15)&(data_f['DDEPT']>0)), 'RIG_STATE'] = 1
#             data_f.loc[((data_f['RIG_STATE'].isna())&(data_f['FLWI']<=15)&(data_f['RPM']<=15)&(data_f['DDEPT']==0)), 'RIG_STATE'] = 2
#             data_f.loc[((data_f['RIG_STATE'].isna())&(data_f['FLWI']>15)&(data_f['RPM']<=15)&(data_f['DDEPT']>0)), 'RIG_STATE'] = 4
#             data_f.loc[((data_f['RIG_STATE'].isna())&(data_f['FLWI']<=15)&(data_f['RPM']>15)&(data_f['DDEPT']>0)), 'RIG_STATE'] = 5
#             data_f.loc[((data_f['RIG_STATE'].isna())&(data_f['FLWI']<=15)&(data_f['RPM']<=15)&(data_f['DDEPT']>0)), 'RIG_STATE'] = 6
#             data_f.loc[((data_f['RIG_STATE'].isna())&(data_f['FLWI']>15)&(data_f['RPM']>15)&(data_f['DDEPT']<0)), 'RIG_STATE'] = 7
#             data_f.loc[((data_f['RIG_STATE'].isna())&(data_f['FLWI']>15)&(data_f['RPM']<=15)&(data_f['DDEPT']<0)), 'RIG_STATE'] = 8
#             data_f.loc[((data_f['RIG_STATE'].isna())&(data_f['FLWI']<=15)&(data_f['RPM']>15)&(data_f['DDEPT']<0)), 'RIG_STATE'] = 9
#             data_f.loc[((data_f['RIG_STATE'].isna())&(data_f['FLWI']<=15)&(data_f['RPM']<=15)&(data_f['DDEPT']<0)), 'RIG_STATE'] = 10
#             data_f.loc[((data_f['RIG_STATE'].isna())&(data_f['FLWI']>15)&(data_f['RPM']>15)&(data_f['DDEPT']==0)), 'RIG_STATE'] = 11
#             data_f.loc[((data_f['RIG_STATE'].isna())&(data_f['FLWI']>15)&(data_f['RPM']<=15)&(data_f['DDEPT']==0)), 'RIG_STATE'] = 12
#             data_f.loc[((data_f['RIG_STATE'].isna())&(data_f['FLWI']<=15)&(data_f['RPM']>15)&(data_f['DDEPT']==0)), 'RIG_STATE'] = 13
#         data_f['RIG_STATE'] = data_f['RIG_STATE'].fillna(method='bfill')
        
#         if hole == '279':
#             data_f = data_f.fillna(0)
#             data_f.loc[(data_f['RIG_STATE']!=2), 'BPOS'] = -data_f.loc[data_f['RIG_STATE']!=2]['DDEPT'].cumsum()
        
#         if hole == '256ST3':
#             data_f['STOR'] = data_f['STOR'].interpolate()
            
#         data_f['BPOS'] = data_f['BPOS'].fillna(method='bfill')
#         data_f['BPOS'] = data_f['BPOS'].fillna(method='ffill')
        
#         data_f['HKLD'].fillna((data_f['HKLD'].mean()), inplace=True)
#         data_f['STOR'].fillna((data_f['STOR'].mean()), inplace=True)
        
#         FLWI_0_RIG = [2, 5, 6, 9, 10, 13, 14] # Список статусов RIG_STATE, при которых FLWI = 0
#         RPM_0_RIG = [2, 4, 6, 8, 10, 12, 14] # То же для RPM = 0
#         MOVE_RIG = [1, 3, 4, 5, 6, 7, 8, 9, 10] # Список статусов RIG_STATE, при которых крюк двигается
        
#         data_f.loc[((data_f['FLWI'].isna())&(data_f['RIG_STATE'].isin(FLWI_0_RIG))), 'FLWI'] = 0
#         data_f.loc[data_f['FLWI'].isna(), 'FLWI'] = data_f['FLWI'].interpolate()
#         data_f.loc[((data_f['RPM'].isna())&(data_f['RIG_STATE'].isin(RPM_0_RIG))), 'RPM'] = 0
#         data_f.loc[data_f['RPM'].isna(), 'RPM'] = data_f['RPM'].interpolate()
#         data_f['SPPA'] = data_f['SPPA'].interpolate()
#         data_f['ECD'] = data_f['ECD'].fillna(method='ffill')
#         data_f['ECD'] = data_f['ECD'].fillna(method='bfill')
        
#         data_f['DLS'] = data_f['DLS'].fillna(method='ffill')
#         data_f['DLS'] = data_f['DLS'].fillna(method='bfill')
#         if 'INCL' in data_f.columns:
#             data_f[['INCL', 'AZIM']] = data_f[['INCL', 'AZIM']].interpolate()
#             data_f[['INCL', 'AZIM']] = data_f[['INCL', 'AZIM']].fillna(method='bfill')
#         if 'APRS' in data_f.columns:
#             data_f['APRS'] = data_f['APRS'].fillna(method='ffill')
#             data_f['APRS'] = data_f['APRS'].fillna(method='bfill')
#         data_f.loc[((data_f['BVEL'].isna())&(data_f['RIG_STATE'].isin(MOVE_RIG))), 'BVEL'] = data_f.loc[((data_f['BVEL'].isna())&(data_f['RIG_STATE'].isin(MOVE_RIG)))]['DDEPT']/360
#         data_f['BVEL'] = data_f['BVEL'].fillna(0)
#         join_Stick(data_f)
#         data_f['Stick'] = data_f.Stick.fillna(0)
#         data_f['ESD'] = data_f['ESD'].fillna(method='ffill')
#         data_f['ESD'] = data_f['ESD'].fillna(method='bfill')
#         if 'GR' in data_f.columns:
#             data_f['GR'] = data_f['GR'].fillna(method='bfill')
#             data_f['GR'] = data_f['GR'].fillna(method='ffill')
        
#         # новые признаки по временному сдвигу
#         data_f['DDEPT_1'] = data_f['DEPT'].diff()/data_f['DEPT'].shift()
#         data_f['DDEPT_3'] = data_f['DEPT'].diff(3)/data_f['DEPT'].shift(3)
#         data_f['DDEPT_6'] = data_f['DEPT'].diff(6)/data_f['DEPT'].shift(6)
#         data_f['DDEPT_12'] = data_f['DEPT'].diff(12)/data_f['DEPT'].shift(12)
#         data_f['DDEPT_18'] = data_f['DEPT'].diff(18)/data_f['DEPT'].shift(18)
#         data_f['FDEPT_1'] = data_f['DEPT'].diff(-1)/data_f['DEPT'].shift(-1)
#         data_f['FDEPT_3'] = data_f['DEPT'].diff(-3)/data_f['DEPT'].shift(-3)
        
#         data_f.loc[data_f['RIG_STATE']==2, 'BPOS'] = 1
#         data_f.loc[data_f['RIG_STATE']!=2, 'BPOS'] = -data_f['DDEPT'].cumsum()

#         data_f['DBPOS'] = data_f['BPOS'].diff() # Новый признак изменения к предыдущему высоты крюка
#         data_f['DBPOS_1'] = data_f['BPOS'].diff()/data_f['BPOS'].shift()
#         data_f['DBPOS_3'] = data_f['BPOS'].diff(3)/data_f['BPOS'].shift(3)
#         data_f['DBPOS_6'] = data_f['BPOS'].diff(6)/data_f['BPOS'].shift(6)
#         data_f['DBPOS_12'] = data_f['BPOS'].diff(12)/data_f['BPOS'].shift(12)
#         data_f['DBPOS_18'] = data_f['BPOS'].diff(18)/data_f['BPOS'].shift(18)
#         data_f['FBPOS_1'] = data_f['BPOS'].diff(-1)/data_f['BPOS'].shift(-1)
#         data_f['FBPOS_3'] = data_f['BPOS'].diff(-3)/data_f['BPOS'].shift(-3)
    
#         data_f['DHKLD'] = data_f['HKLD'].diff()# Новый признак изменения к предыдущему веса крюка
#         data_f['DHKLD_1'] = data_f['HKLD'].diff()/data_f['HKLD'].shift()
#         data_f['DHKLD_3'] = data_f['HKLD'].diff(3)/data_f['HKLD'].shift(3)
#         data_f['DHKLD_6'] = data_f['HKLD'].diff(6)/data_f['HKLD'].shift(6)
#         data_f['DHKLD_12'] = data_f['HKLD'].diff(12)/data_f['HKLD'].shift(12)
#         data_f['DHKLD_18'] = data_f['HKLD'].diff(18)/data_f['HKLD'].shift(18)
#         data_f['FHKLD_1'] = data_f['HKLD'].diff(-1)/data_f['HKLD'].shift(-1)
#         data_f['FHKLD_3'] = data_f['HKLD'].diff(-3)/data_f['HKLD'].shift(-3)
      
#         data_f['DSPPA'] = data_f['SPPA'].diff()
#         data_f['DSPPA_1'] = data_f['SPPA'].diff()/data_f['SPPA'].shift()
#         data_f['DSPPA_3'] = data_f['SPPA'].diff(3)/data_f['SPPA'].shift(3)
#         data_f['DSPPA_6'] = data_f['SPPA'].diff(6)/data_f['SPPA'].shift(6)
#         data_f['DSPPA_12'] = data_f['SPPA'].diff(12)/data_f['SPPA'].shift(12)
#         data_f['DSPPA_18'] = data_f['SPPA'].diff(18)/data_f['SPPA'].shift(18)
#         data_f['FSPPA_1'] = data_f['SPPA'].diff(-1)/data_f['SPPA'].shift(-1)
#         data_f['FSPPA_3'] = data_f['SPPA'].diff(-3)/data_f['SPPA'].shift(-3)
        
#         data_f['DRPM'] = data_f['RPM'].diff()
#         data_f['DRPM_1'] = data_f['RPM'].diff()/data_f['RPM'].shift()
#         data_f['DRPM_3'] = data_f['RPM'].diff(3)/data_f['RPM'].shift(3)
#         data_f['DRPM_6'] = data_f['RPM'].diff(6)/data_f['RPM'].shift(6)
#         data_f['DRPM_12'] = data_f['RPM'].diff(12)/data_f['RPM'].shift(12)
#         data_f['DRPM_18'] = data_f['RPM'].diff(18)/data_f['RPM'].shift(18)
#         data_f['FRPM_1'] = data_f['RPM'].diff(-1)/data_f['RPM'].shift(-1)
#         data_f['FRPM_3'] = data_f['RPM'].diff(-3)/data_f['RPM'].shift(-3)

#         data_f['DBVEL'] = data_f['BVEL'].diff()
#         data_f['DBVEL_1'] = data_f['BVEL'].diff()/data_f['BVEL'].shift()
#         data_f['DBVEL_3'] = data_f['BVEL'].diff(3)/data_f['BVEL'].shift(3)
#         data_f['DBVEL_6'] = data_f['BVEL'].diff(6)/data_f['BVEL'].shift(6)
#         data_f['DBVEL_12'] = data_f['BVEL'].diff(12)/data_f['BVEL'].shift(12)
#         data_f['DBVEL_18'] = data_f['BVEL'].diff(18)/data_f['BVEL'].shift(18)
#         data_f['FBVEL_1'] = data_f['BVEL'].diff(-1)/data_f['BVEL'].shift(-1)
#         data_f['FBVEL_3'] = data_f['BVEL'].diff(-3)/data_f['BVEL'].shift(-3)

#         data_f['DSTOR_1'] = data_f['STOR'].diff()/data_f['STOR'].shift()
#         data_f['DSTOR_3'] = data_f['STOR'].diff(3)/data_f['STOR'].shift(3)
#         data_f['DSTOR_6'] = data_f['STOR'].diff(6)/data_f['STOR'].shift(6)
#         data_f['DSTOR_12'] = data_f['STOR'].diff(12)/data_f['STOR'].shift(12)
#         data_f['DSTOR_18'] = data_f['STOR'].diff(18)/data_f['STOR'].shift(18)
#         data_f['FSTOR_1'] = data_f['STOR'].diff(-1)/data_f['STOR'].shift(-1)
#         data_f['FSTOR_3'] = data_f['STOR'].diff(-3)/data_f['STOR'].shift(-3)
        
#         if hole == '216ST2':
#             data_f['SPPA_APRS'] = 0
#         if hole != '216ST2':
#             data_f['SPPA_APRS'] = data_f['SPPA']/data_f['APRS']
#         data_f['DSPPA_APRS_1'] = data_f['SPPA_APRS'].diff()/data_f['SPPA_APRS'].shift()
#         data_f['DSPPA_APRS_3'] = data_f['SPPA_APRS'].diff(3)/data_f['SPPA_APRS'].shift(3)
#         data_f['DSPPA_APRS_6'] = data_f['SPPA_APRS'].diff(6)/data_f['SPPA_APRS'].shift(6)
#         data_f['DSPPA_APRS_12'] = data_f['SPPA_APRS'].diff(12)/data_f['SPPA_APRS'].shift(12)
#         data_f['DSPPA_APRS_18'] = data_f['SPPA_APRS'].diff(18)/data_f['SPPA_APRS'].shift(18)
#         data_f['FSPPA_APRS_1'] = data_f['SPPA_APRS'].diff(-1)/data_f['SPPA_APRS'].shift(-1)
#         data_f['FSPPA_APRS_3'] = data_f['SPPA_APRS'].diff(-3)/data_f['SPPA_APRS'].shift(-3)
        
#         data_f['DECD'] = data_f['ECD'].diff()
        
#         data_f[['DDEPT', 'DBPOS', 'DHKLD', 'DSPPA', 'DECD']] = data_f[['DDEPT', 'DBPOS', 'DHKLD', 'DSPPA', 'DECD']].fillna(0)
#         shift_list = [
#               'DDEPT_1', 'DDEPT_3', 'DDEPT_6', 'DDEPT_12', 'DDEPT_18', 'FDEPT_1', 'FDEPT_3',
#               'DBPOS_1', 'DBPOS_3', 'DBPOS_6', 'DBPOS_12', 'DBPOS_18', 'FBPOS_1', 'FBPOS_3',
#               'DHKLD_1', 'DHKLD_3', 'DHKLD_6', 'DHKLD_12', 'DHKLD_18', 'FHKLD_1', 'FHKLD_3',
#               'DSPPA_1', 'DSPPA_3', 'DSPPA_6', 'DSPPA_12', 'DSPPA_18', 'FSPPA_1', 'FSPPA_3',
#               'DSTOR_1', 'DSTOR_3', 'DSTOR_6', 'DSTOR_12', 'DSTOR_18', 'FSTOR_1', 'FSTOR_3',
#               'DRPM_1', 'DRPM_3', 'DRPM_6', 'DRPM_12', 'DRPM_18', 'FRPM_1', 'FRPM_3',
#               'DBVEL_1', 'DBVEL_3', 'DBVEL_6', 'DBVEL_12', 'DBVEL_18', 'FBVEL_1', 'FBVEL_3',
#               'DSPPA_APRS_1', 'DSPPA_APRS_3', 'DSPPA_APRS_6', 'DSPPA_APRS_12', 'DSPPA_APRS_18',
#               'FSPPA_APRS_1', 'FSPPA_APRS_3'
#               ]
#         for feat in shift_list:
#             data_f[feat] = data_f[feat].fillna(0)
        
        
#         if n == 0:
#             data=data_f
#             n += 1
#         else:
#             data = pd.concat([data, data_f], axis=0, ignore_index=True)

## 3. Импорт данных

In [ ]:
# Импорт с google.disc при помощи PyDrive

#link - ссылка на файл
#file_name - имя скачиваемого файла excel
def pydrive(link, file_name, file_type='xslx', skiprows=None):
    id = link.split('/')[5]
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile(file_name)
    if file_type=='xslx':
        return pd.read_excel(file_name, skiprows=skiprows)
    elif file_type=='csv':
        return pd.read_csv(file_name, skiprows=skiprows)

In [ ]:
# Импорт с локального компьютера

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving final_data_28_05(180).csv to final_data_28_05(180).csv
User uploaded file "final_data_28_05(180).csv" with length 41016151 bytes


In [ ]:
data = pd.read_csv('final_data_28_05(180).csv')

In [ ]:
print(f'Размерность датасета: {data.shape}')

Размерность датасета: (32774, 92)


## 5. Предварительный анализ данных

### 5.3. Замена нулевых значений

In [ ]:
# Сводная таблица нулевых значений по скважинам
data['count'] = np.nan
df = data.groupby('hole').agg(lambda x: x.isna().sum())
display(df)

,Date/Time,StuckPipe,TVD,DEPT,CDEPTH,HDTH,BPOS,HKLD,STOR,FLWI,RPM,SPPA,ECD,DLS,INCL,AZIM,GR,APRS,BVEL,RIG_STATE,Stick_Slip_Ratio,StickPercentage,ESD,file,DDEPT,...,DBVEL_3,DBVEL_6,DBVEL_12,DBVEL_18,FBVEL_1,FBVEL_3,DSTOR_1,DSTOR_3,DSTOR_6,DSTOR_12,DSTOR_18,FSTOR_1,FSTOR_3,SPPA_APRS,DSPPA_APRS_1,DSPPA_APRS_3,DSPPA_APRS_6,DSPPA_APRS_12,DSPPA_APRS_18,FSPPA_APRS_1,FSPPA_APRS_3,DECD,DataFilter,Unnamed: 23,count
hole,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
279,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0,6.0
204,0,0.0,4132.0,0.0,4132.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,4132.0,2033.0,2033.0,0.0,0.0,0.0,0.0,4132.0,3862.0,1295.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4132.0,4132.0,4132.0
210ST,0,0.0,0.0,0.0,1117.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1117.0,190.0,190.0,0.0,0.0,0.0,0.0,1051.0,1052.0,187.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1117.0,1117.0,1117.0
210ST2,0,0.0,1911.0,0.0,16.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1826.0,1911.0,826.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1911.0,1911.0,1911.0
216,0,0.0,610.0,0.0,610.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,217.0,217.0,217.0,0.0,0.0,0.0,0.0,588.0,588.0,0.0,404,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,610.0,610.0,610.0
216ST2,0,0.0,0.0,0.0,740.0,0.0,4.0,740.0,0.0,0.0,0.0,0.0,740.0,740.0,740.0,740.0,740.0,740.0,0.0,0.0,740.0,740.0,740.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,740.0,740.0,740.0
217ST1,0,0.0,680.0,0.0,0.0,0.0,2.0,0.0,680.0,0.0,0.0,0.0,0.0,0.0,680.0,302.0,0.0,0.0,0.0,0.0,680.0,680.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,680.0,680.0,680.0
217ST2,0,0.0,0.0,0.0,1002.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1002.0,194.0,0.0,0.0,0.0,0.0,1002.0,972.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1002.0,1002.0,1002.0
220,0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1443.0,1375.0,411.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1443.0,1443.0,1443.0


**Выводы:**
1. По скважинам №255 и 216ST2 основные параметры отстутсвуют в пределах 75-100%. Предлагаю удалить эти скважины из обучающей выборки.

In [ ]:
data = data.loc[~(data['hole']=='255')]
data = data.loc[~(data['hole']=='216ST2')]
data = data.loc[~(data['hole']==279)]

In [ ]:
data['STOR'].fillna((data['STOR'].mean()), inplace=True)

In [ ]:
data['ECD'] = data['ECD'].fillna(method='ffill')
data['ECD'] = data['ECD'].fillna(method='bfill')

In [ ]:
data[['DLS', 'INCL', 'AZIM']] = data[['DLS', 'INCL', 'AZIM']].fillna(0)

In [ ]:
data['GR'].fillna((data['GR'].mean()), inplace=True)

In [ ]:
data['APRS'].fillna((data['APRS'].mean()), inplace=True)

In [ ]:
data['ESD'].fillna((data['ESD'].mean()), inplace=True)

In [ ]:
data = data.fillna(method='bfill')

### 5.4. Анализ на выбросы и их обработка

In [ ]:
# Загрузим и наполним таблицу минимальными и максимальными значениями датасета и выставим флаг по столбцам с выбросами

df_range = pydrive('https://drive.google.com/file/d/1wp1LY_9FKr5R1mkFIEB2xtrcP7sq5qwL/view?usp=sharing', 'range.csv', file_type='csv')
df_r = pd.pivot_table(df_range, index=["RANGE"])
df_r['min_real']=0
df_r['max_real']=0

In [ ]:
#df_r = df_r.drop(['Stick'], axis=0)
df_r = df_r.drop(['CDEPTH', 'StickPercentage', 'Stick_Slip_Ratio', 'TVD' ], axis=0)

for rows in df_r.index:
    df_r['min_real'][rows]= data[rows].min()
    df_r['max_real'][rows]= data[rows].max()
    
df_r['discard'] = df_r.apply(lambda n: 1 if n['max'] < n['max_real'] or n['min'] > n['min_real'] else 0, axis=1)


In [ ]:
# Создадим признак аномальных значений по атрибутам ['GR', 'BPOS', 'RPM', 'SPPA', 'STOR', 'ECD', 'INCL', 'APRS', 'Stick']
# Создание признака возможно после заполнения пропущенных значений!!!

data['ANOMAL'] = 0
for rows in  ['GR', 'BPOS', 'RPM', 'SPPA', 'STOR', 'ECD', 'INCL', 'APRS', 'Stick']: # Пока еще у нас нет 'Stick', я опустил feature engineering ниже
    data.loc[data[rows]<df_r['min'][rows], 'ANOMAL'] += 1
    data.loc[data[rows]>df_r['max'][rows], 'ANOMAL'] += 1

In [ ]:
data['ANOMAL'].value_counts()

0    21081
1     5374
2     1840
3      683
5       49
4       18
Name: ANOMAL, dtype: int64

In [ ]:
# Приведем значения выбросов к значениям с отклонением на 10% от граничных значений
cat_cols = ['StuckPipe', 'RIG_STATE', 'hole', 'file_name', 'Date/Time']
num_cols = list(set(df_r.index) - set(cat_cols))

for col in num_cols:
    data.loc[data[col]>df_r['max'][col]*1.5, col] = df_r['max'][col]*1.5
 #   data.loc[data[col]<df_r['min'][col], col] = -15

#data.loc[data['Stick']>df_r['max']['Stick']*1.5, 'Stick'] = df_r['max']['Stick']*1.5
data.loc[data['Stick']<(-300), 'Stick'] = -300

# data[data['BVEL'] > df_r['max']['BVEL']*1.5]['BVEL'] = df_r['max']['BVEL']*1.1
# data[data['BVEL'] < df_r['min']['BVEL']*1.1] = df_r['min']['BVEL']*1.1 #здесь ошибка, все значения строки изменятся вместо одного

# data[data['Stick'] > df_r['max']['Stick']*1.1]['Stick']= df_r['max']['Stick']*1.1
# data[data['Stick'] < df_r['min']['Stick']*1.1]['Stick'] = 0

### 5.5. Генерация новых признаков

In [ ]:
# Приведем время к unix формату

data['Date/Time'] = pd.to_datetime(data['Date/Time'])
data['TimeU'] = data['Date/Time'].apply(lambda t: int(t.timestamp()))
# data = data.drop(['Date/Time'], axis=1)

In [ ]:
# Генерация новых признаков - разница между выбросом и пределом

discard_list = ['SPPA', 'STOR', 'ECD', 'Stick', 'APRS', 'RPM']
for rows in discard_list:
    IQR = data[rows].quantile(0.75) - data[rows].quantile(0.25)
    perc25 = data[rows].quantile(0.25)
    perc75 = data[rows].quantile(0.75)
    f = perc25 - 1.5*IQR
    l = perc75 + 1.5*IQR
    print(rows, '25-й перцентиль: {},'.format(perc25), '75-й перцентиль: {},'.format(perc75)
          , "IQR: {}, ".format(IQR),"Границы выбросов: [{}, {}].".format(f, l))
    data[str('discard'+rows)] = 0
    #data.loc[data[rows]<f, str('discard'+rows)] = data[rows].apply(lambda r: (f-r))
    data.loc[data[rows]>df_r['max'][rows], str('discard'+rows)] = data[rows].apply(lambda r: (r - df_r['max'][rows]))
    data.loc[data[rows]<0, str('discard'+rows)] = data[rows].apply(lambda r: r)

SPPA 25-й перцентиль: 17.924, 75-й перцентиль: 187.735284, IQR: 169.811284,  Границы выбросов: [-236.792926, 442.45221000000004].
STOR 25-й перцентиль: 5.684974100719423, 75-й перцентиль: 21.553565, IQR: 15.868590899280576,  Границы выбросов: [-18.117912248201442, 45.356451348920864].
ECD 25-й перцентиль: 1.4008, 75-й перцентиль: 1.5661, IQR: 0.1653,  Границы выбросов: [1.15285, 1.81405].
Stick 25-й перцентиль: 0.0, 75-й перцентиль: 0.0, IQR: 0.0,  Границы выбросов: [0.0, 0.0].
APRS 25-й перцентиль: 156.30415000000005, 75-й перцентиль: 223.63836600000002, IQR: 67.33421599999997,  Границы выбросов: [55.302826000000096, 324.63969].
RPM 25-й перцентиль: 11.43, 75-й перцентиль: 100.42666666666666, IQR: 88.99666666666667,  Границы выбросов: [-122.065, 233.92166666666668].


In [ ]:
# Новый признак отношения расстояния до долота к общей длине скважины

data['PDEPT']=data['DEPT']/data['HDTH']

In [ ]:
# Новый признак содержащий примерное отношение к формуле расчета силы давления на дно и стенки сосуда F= плотность*g*H*S
# Зависимость GR и плотности обратная - чем больше Градусов API тем меньше плотность в кг/м3, поэтому берем 1/GR
# http://oilreview.kiev.ua/tpga-mup/
# Добавим признак, учитывающий угол наклона поверхности при давлении на дно.
# При корреляции выберем один из признаков потом

data['F'] = data['DEPT']/data['GR']

data['F_all'] = data['F']*data['INCL'].apply(lambda u: np.sin(u))
data['F_all'] = data['F_all'].fillna(method='bfill')

In [ ]:
#Новый признак - 'F_all' - вспомогательный, потом удалить
# INCL - зенитный угол, отклонение касательной ствола скважины от вертикали (градус);
# DLS - интенсивность искривления скважины (30 м от забоя по HDTH) (градус/30 м);
# AZIM - бесполезная информация в данной задаче, поскольку показывает только направление по отношению к Северу по поверхности а не глубине
# сумма INCL + DLS по идее должна быть в пределах 90 градусов. Если то больше 90, то меньше, то это образуется некое "вихляние" скважины
# сыделаем из этого отдельный признак

display(data['INCL'].unique())
display(data['DLS'].unique())
display(data['AZIM'].unique())

data['conner'] = data['INCL'] + data['DLS']
data['conner_delta'] =  data['conner'].apply(lambda u: 1 if u > 90 else 0)


array([85.79      , 85.78978261, 85.78956522, ..., 81.79      ,
       81.8       , 81.35      ])

array([0.    , 2.3819, 2.5   , 2.3228, 2.3917, 1.7913, 2.9035, 3.5138,
       2.8543, 3.376 , 3.7205, 1.5059, 3.878 , 1.7421, 1.4173, 2.5984,
       2.8346, 2.3622, 1.2598, 0.1673, 0.374 , 0.3839, 0.2362, 0.2165,
       0.8169, 0.2657, 0.6791, 0.315 , 0.3051, 0.4429, 2.2539, 1.9587,
       1.7618, 3.3071, 1.1614, 1.1417, 1.6043, 1.7224, 1.624 , 1.3878,
       2.2638, 2.8248, 3.1201, 3.5236, 2.0177, 1.752 , 2.4508, 2.6673,
       2.1457, 1.4961, 2.9823, 4.3602, 3.2874, 4.8524, 2.6772, 1.437 ,
       0.5906, 2.7362, 2.7756, 2.1555, 1.998 , 0.1083, 0.128 , 0.0492,
       0.1575, 0.1772, 0.3543, 0.2756, 0.6004, 0.2067, 2.5689, 2.6969,
       2.2244, 2.7067, 2.5591, 0.5315, 0.1969, 0.3346, 0.5118, 0.502 ,
       0.1476, 0.2461, 0.0295, 0.4035, 0.3445, 0.6201, 0.2559, 0.1378,
       0.0886, 0.2264])

array([ 18.77      ,  18.77141304,  18.77282609, ..., 100.5       ,
       100.51      , 100.52      ])

In [ ]:
# Давление раствора на забое APRS складывается из давления насосов SPPA и гидростатического давления. Гидростатическое давление - 
# давление столба жидкости на дно сосуда (зависит от плотности бурового раствора ESD и абсолютной глубины ствола TVD).
# Идея: Принимаем гидростатическое давление за const. Снижение APRS относительно SPPA может свидетельствовать об обрушении стенок скважины и возрастания вероятности аварии на обратной проработке.
# Вывод: Вводим новую переменную как отношение APRS к SPPA.

# data['SPPA_APRS'] = data['SPPA']/data['APRS'] # Вывел в считывание файлов
data['SPPA_APRS'] = data['SPPA_APRS'].fillna(0)

In [ ]:
# GR - естественная радиоактивность пород в скважине. Значительной радоактивностью обладают глины >200. Песчаники наоборот, имеют низкую <60.
data['GR_type'] = 1
data.loc[data['GR'] < 60, 'GR_type'] = 0
data.loc[data['GR'] > 200, 'GR_type'] = 2
data['GR_type'].value_counts()

1    23651
0     4636
2      758
Name: GR_type, dtype: int64

In [ ]:
#Бинарные переменные по наличию выбросов на интервале
# Вылавливаются выбросы и ставится единица, если больше или меньше квантилей.
# Так как разброс по скважинам отличается, то сделал в разрезе скважин
# Корреляция с целевой переменной найдена по следующим переменным
bin_dict = {
    'FDEPT_3': ['DEPT_bin', 0.99, 0.05], 
    'FSPPA_1': ['SPPA_bin', 0.99, 0.05], 
    'FSPPA_APRS_1': ['SPPA_APRS_bin', 0.99, 0.05], 
    'FBVEL_3': ['BVEL_bin', 0.97, 0.05], 
    'FHKLD_3': ['HKLD_bin', 0.94, 0.01], 
    'FBPOS_3': ['BPOS_bin', 0.89, 0.01], 
    'FRPM_3': ['RPM_bin', 0.97, 0.02], 
    'FSTOR_3': ['STOR_bin', 0.98, 0.05]
    }
for val in bin_dict.values():
        data[val[0]] = 0

for hole in data.hole.unique():
    for key, val in bin_dict.items():
        df1 = data.loc[(data['hole']==hole) & (data['RIG_STATE']==7)]
        df1.loc[(df1[key] >= df1[key].quantile(val[1])), val[0]] = 1
        df1.loc[(df1[key] <= df1[key].quantile(val[2])), val[0]] = 1
        data.loc[data.index.isin(df1.loc[df1[val[0]]==1].index), [val[0]]] = 1

In [ ]:
# Те же самые бинарные переменные, но рассчет производится для каждой скважины отдельно
bin_dict_new = {
    'FDEPT_3': 'DEPT_bin_new', 
    'FSPPA_1': 'SPPA_bin_new', 
    'FSPPA_APRS_1': 'SPPA_APRS_bin_new', 
    'FBVEL_3': 'BVEL_bin_new', 
    'FHKLD_3': 'HKLD_bin_new', 
    'FBPOS_3': 'BPOS_bin_new', 
    'FRPM_3': 'RPM_bin_new', 
    'FSTOR_3': 'STOR_bin_new'
    }

for val in bin_dict_new.values():
        data[val] = 0

hole_dict = {}

for hole in data.hole.unique():
    hole_dict[hole] = {}
    for key, val in bin_dict_new.items():
        new_dict = {}
        hole_dict[hole][key] = [val]
        for i in range(89, 100):
            for j in range(1, 8):
                df1 = data.loc[(data['hole']==hole) & (data['RIG_STATE']==7)]
                df1.loc[(df1[key] >= df1[key].quantile(i/100)), val] = 1
                df1.loc[(df1[key] <= df1[key].quantile(j/100)), val] = 1
                if np.isnan(df1[val].corr(df1['StuckPipe'])):
                    new_dict[0]=[i/100, j/100]
                else:
                    new_dict[abs(df1[val].corr(df1['StuckPipe']))]=[i/100, j/100]
        max_corr = max(new_dict.keys())
        hole_dict[hole][key].append(new_dict[max_corr])


for hole, value in hole_dict.items():
    for key, val in value.items():
        df2 = data.loc[(data['hole']==hole) & (data['RIG_STATE']==7)]
        df2.loc[(df2[key] >= df2[key].quantile(val[1][0])), val[0]] = 1
        df2.loc[(df2[key] <= df2[key].quantile(val[1][1])), val[0]] = 1
        data.loc[data.index.isin(df2.loc[df2[val[0]]==1].index), [val[0]]] = 1

In [ ]:
# Те же самые бинарные переменные, но рассчет производится на основе исторических данных без заглядывания вперед (30 сек)
bin_dict_new = {
    'DDEPT_3': 'DEPT_back3', 
    'DSPPA_3': 'SPPA_back3', 
    'DSPPA_APRS_3': 'SPPA_APRS_back3', 
    'DBVEL_3': 'BVEL_back3', 
    'DHKLD_3': 'HKLD_back3', 
    'DBPOS_3': 'BPOS_back3', 
    'DRPM_3': 'RPM_back3', 
    'DSTOR_3': 'STOR_back3'
    }

for val in bin_dict_new.values():
        data[val] = 0

hole_dict = {}

for hole in data.hole.unique():
    hole_dict[hole] = {}
    for key, val in bin_dict_new.items():
        new_dict = {}
        hole_dict[hole][key] = [val]
        for i in range(89, 100):
            for j in range(1, 8):
                df1 = data.loc[(data['hole']==hole) & (data['RIG_STATE']==7)]
                df1.loc[(df1[key] >= df1[key].quantile(i/100)), val] = 1
                df1.loc[(df1[key] <= df1[key].quantile(j/100)), val] = 1
                if np.isnan(df1[val].corr(df1['StuckPipe'])):
                    new_dict[0]=[i/100, j/100]
                else:
                    new_dict[abs(df1[val].corr(df1['StuckPipe']))]=[i/100, j/100]
        max_corr = max(new_dict.keys())
        hole_dict[hole][key].append(new_dict[max_corr])


for hole, value in hole_dict.items():
    for key, val in value.items():
        df2 = data.loc[(data['hole']==hole) & (data['RIG_STATE']==7)]
        df2.loc[(df2[key] >= df2[key].quantile(val[1][0])), val[0]] = 1
        df2.loc[(df2[key] <= df2[key].quantile(val[1][1])), val[0]] = 1
        data.loc[data.index.isin(df2.loc[df2[val[0]]==1].index), [val[0]]] = 1

In [ ]:
# Те же самые бинарные переменные, но рассчет производится на основе исторических данных без заглядывания вперед (10 сек)
bin_dict_new = {
    'DDEPT_1': 'DEPT_back1', 
    'DSPPA_1': 'SPPA_back1', 
    'DSPPA_APRS_1': 'SPPA_APRS_back1', 
    'DBVEL_1': 'BVEL_back1', 
    'DHKLD_1': 'HKLD_back1', 
    'DBPOS_1': 'BPOS_back1', 
    'DRPM_1': 'RPM_back1', 
    'DSTOR_1': 'STOR_back1'
    }

for val in bin_dict_new.values():
        data[val] = 0

hole_dict = {}

for hole in data.hole.unique():
    hole_dict[hole] = {}
    for key, val in bin_dict_new.items():
        new_dict = {}
        hole_dict[hole][key] = [val]
        for i in range(89, 100):
            for j in range(1, 8):
                df1 = data.loc[(data['hole']==hole) & (data['RIG_STATE']==7)]
                df1.loc[(df1[key] >= df1[key].quantile(i/100)), val] = 1
                df1.loc[(df1[key] <= df1[key].quantile(j/100)), val] = 1
                if np.isnan(df1[val].corr(df1['StuckPipe'])):
                    new_dict[0]=[i/100, j/100]
                else:
                    new_dict[abs(df1[val].corr(df1['StuckPipe']))]=[i/100, j/100]
        max_corr = max(new_dict.keys())
        hole_dict[hole][key].append(new_dict[max_corr])


for hole, value in hole_dict.items():
    for key, val in value.items():
        df2 = data.loc[(data['hole']==hole) & (data['RIG_STATE']==7)]
        df2.loc[(df2[key] >= df2[key].quantile(val[1][0])), val[0]] = 1
        df2.loc[(df2[key] <= df2[key].quantile(val[1][1])), val[0]] = 1
        data.loc[data.index.isin(df2.loc[df2[val[0]]==1].index), [val[0]]] = 1

In [ ]:
data = data.drop(['TVD', 'CDEPTH', 'Stick_Slip_Ratio', 'StickPercentage', 'file', 'DataFilter', 'Unnamed: 23', 'count'], axis=1)

In [ ]:
data = data.replace([np.inf, -np.inf], np.nan).fillna(0)

In [ ]:
data.shape

(29045, 139)

In [ ]:
data.to_csv('all_eda_obr28_05(180).csv')
files.download('all_eda_obr28_05(180).csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>